In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain_openai import ChatOpenAI

In [4]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [5]:
KEY=os.getenv("OPENAI_API_KEY")


In [6]:
print(KEY)

sk-proj-MR14ClNnarlE0mQcEDGTT3BlbkFJr0Q6Z1g5ltpDlDZVnkHq


In [7]:
llm=ChatOpenAI(openai_api_key=KEY,tiktoken_model_name="gpt-3.5-turbo", temperature=0.5)

In [8]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11e50f850>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12404b890>, temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='', tiktoken_model_name='gpt-3.5-turbo')

In [9]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [10]:
RESPONSE_JSON = {
    "1": { 
        "mcq": "multiple choise question",
        "options": {
                "a": "choice here",
                "b": "choice here",
                "c": "choice here",
                "d": "choice here",
            },
            "correct": "correct answer",
            },
    "2": { 
        "mcq": "multiple choise question",
        "options": {
                "a": "choice here",
                "b": "choice here",
                "c": "choice here",
                "d": "choice here",
            },
            "correct": "correct answer",
            },
    "3": { 
        "mcq": "multiple choise question",
        "options": {
                "a": "choice here",
                "b": "choice here",
                "c": "choice here",
                "d": "choice here",
            },
            "correct": "correct answer",
            },
}

In [11]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to conforming the text as well.
Make sure to format your responses like RESPONSE_JSON below and use is as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [12]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [13]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

/Users/yulia/mcqgen/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [14]:
TEMPLATE2="""
You are an expert english grammar and writer. Given a multiple Choise Quiz for {subject} students. \
You need to evaluate the complexity of the question and give a complete analisis of the quiz.
Only use at max 50 words for complexity if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz wuestions wich needs to be changed and change the tone such that it perfectly fits the students.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [15]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [16]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [17]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], 
                        input_variables=["text","number", "subject", "tone", "response_json"], 
                        output_variables=["quiz", "review"], 
                        verbose=True)

In [18]:
file_path="/Users/yulia/mcqgen/data.txt"

In [19]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [20]:
print(TEXT)


THE 12 COLOUR SEASONS
Complete Seasonal Guides
Twelve seasons colour analysis comprises four main seasons – Spring, Summer, Autumn and Winter. These are further divided into three sub-seasons each, resulting in twelve colour seasons in total. On the basis of the flow theory, all seasons flow into each other forming a full circle.

In this section, you can learn more about each colour season and access the comprehensive guides.

Spring
Spring is the season of renewal, painting the natural world in fresh, blooming colours. Consequently, Spring features are warm, bright and light, giving Springs an overall luminous and fresh appearance.

The Spring season is divided into three sub-seasons: Bright Spring, True Spring and Light Spring.

True Spring Colour Wheel
Bright + Warm

Bright Spring is the combination of bright and warm. This season flows into Spring from Winter, which tints skin, hair, and eyes neutral-warm. The most prominent characteristic of Bright Springs is the clearness of th

In [21]:
# Serialize the Python Dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choise question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choise question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choise question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [22]:
NUMBER=10
SUBJECT="Color types"
TONE="simple"

In [23]:
# https://python.langchain.com/v0.1/docs/modules/model_io/llms/token_usage_tracking/
# How to setup Tokenn Usagr Tracking in LangChain

with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
        "text":TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
        }
    )

/Users/yulia/mcqgen/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:
THE 12 COLOUR SEASONS
Complete Seasonal Guides
Twelve seasons colour analysis comprises four main seasons – Spring, Summer, Autumn and Winter. These are further divided into three sub-seasons each, resulting in twelve colour seasons in total. On the basis of the flow theory, all seasons flow into each other forming a full circle.

In this section, you can learn more about each colour season and access the comprehensive guides.

Spring
Spring is the season of renewal, painting the natural world in fresh, blooming colours. Consequently, Spring features are warm, bright and light, giving Springs an overall luminous and fresh appearance.

The Spring season is divided into three sub-seasons: Bright Spring, True Spring and Light Spring.

True Spring Colour Wheel
Bright + Warm

Bright Spring is the combination of bright and warm. This season flows into Spring from Winter, which tints s

In [24]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")

Total Tokens: 20897
Prompt Tokens: 19390
Completion Tokens: 1507
Total Cost (USD): $0.032099


In [26]:
quiz=response.get("quiz")

In [29]:
quiz=json.loads(quiz)

In [31]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ". join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [32]:
quiz_table_data

[{'MCQ': 'Which season is known for its warm, bright, and light features?',
  'Choices': 'a: Spring | b: Summer | c: Autumn | d: Winter',
  'Correct': 'a'},
 {'MCQ': 'Which sub-seasons make up the Spring color analysis?',
  'Choices': 'a: Bright Spring, True Spring, Dark Spring | b: Bright Spring, True Spring, Light Spring | c: Bright Spring, Warm Spring, Soft Spring | d: Bright Spring, Cool Spring, Muted Spring',
  'Correct': 'b'},
 {'MCQ': 'What is the primary characteristic of a Bright Spring?',
  'Choices': 'a: Cool and muted | b: Warm and bright | c: Dark and warm | d: Cool and bright',
  'Correct': 'b'},
 {'MCQ': 'Which season is a blend of light and warm features?',
  'Choices': 'a: True Summer | b: Soft Summer | c: Light Summer | d: Bright Summer',
  'Correct': 'c'},
 {'MCQ': 'What is the distinguishing feature of a True Autumn?',
  'Choices': 'a: Bright and cool | b: Muted and warm | c: Dark and cool | d: Warm and muted',
  'Correct': 'b'},
 {'MCQ': 'Which season is known for 

In [34]:
quiz=pd.DataFrame(quiz_table_data)

In [35]:
quiz.to_csv("color_analisys.csv", index=False)